# 项目简介

> 2020.08.01更新：
> 1. 新增使用`WaveFlow`作为声码器的`Transformer TTS`文字转语音实现
> 2. AI Studio已支持libsndfile1，语音合成效果已可以在线验证，修改相关介绍
> 
> 2020.08.03更新：
> 1. 新增基于HTML、PDF、OCR解析的`Fastspeech`文字转语音实现：[Parakeet：飞桨，你是个成熟的框架了，要学会自己读论文](https://aistudio.baidu.com/aistudio/projectdetail/676162)

该项目使用语音合成模型[WaveFlow](https://github.com/PaddlePaddle/Parakeet/blob/develop/examples/waveflow/train.py)作为AI Studio脚本环境的示例任务，同时提供Notebook环境作为对比，**强烈建议读者对照两个项目阅读**，便于理解。

## 演示环境

- [脚本任务示例：Parakeet——WaveFlow语音合成模型单卡训练](https://aistudio.baidu.com/aistudio/clusterprojectdetail/629531)
- 本文：Notebook项目部署与脚本任务对比解读
- ***建议读者***搭建本地环境进一步对比

## 脚本项目介绍

近期AI Studio版本更新之后，脚本任务运行环境最高支持到`Tesla V100*8`！

![file](https://ai-studio-static-online.cdn.bcebos.com/ce5f3505e7d24b71b7e2843ae65d11831690b2eeaddf4e9e882a779783a60602)

![file](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1595260465871&di=5970cebcdda3989a4709bed37623fd2b&imgtype=0&src=http%3A%2F%2F5b0988e595225.cdn.sohucs.com%2Fimages%2F20180404%2F7bc8914d751046f596b1322442f99937.gif)


然后再看一眼算力卡消耗标准，那些用不完的算力有去处了……


| P40                         | V100 4卡                    | V100 8卡                    |
| --------------------------- | --------------------------- | --------------------------- |
| 0.5算力/小时                | 4算力/小时                  | 8算力/小时                  |

![file](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1595260293983&di=3e612d3033d8f0d39fe42062e52cf710&imgtype=0&src=http%3A%2F%2Fp2.ssl.cdn.btime.com%2Ft01a7f1216344770ebe.gif%3Fsize%3D500x375)

然而，平台上目前公开的脚本任务示例不多（算力还是挺紧张的，非闲时需要排队，排了一天都没用到P40），因此，本文选择一个脚本任务开发的典型场景做为示例介绍，希望对大家有所帮助。

## Parakeet模型库介绍

Parakeet是Paddlepaddle近期上新的文字转语音（TTS,Text-to-Speech）工具库，使用飞桨模型库实现端到端的语音合成。如果您使用过各类XX读书或者新版Edge浏览器的朗读功能，这些都是典型的TTS场景。

[Parakeet项目地址](https://github.com/PaddlePaddle/Parakeet)

### WaveFlow模型

本文使用的是百度最新推出的WaveFlow模型，根据官网介绍，只有5.9M参数量，比经典的WaveGlow语音合成模型小了15倍，同时语音合成效果也非常好。WaveFlow和WaveGlow都是基于流的生成模型，它和GAN都属于生成模型家族。

**需要注意的是，WaveFlow是个vocoder（声码器，一种将声学参数转换成语音波形的工具）,不能直接实现文字转语音，需要与Parakeet库中的TTS模型Deep Voice 3、Transformer TTS或FastSpeech模型结合，实现文字转语音的拟声合成。**

关于模型的详细介绍请参考：
- [WaveFlow论文地址](https://arxiv.org/abs/1912.01219)
- [WaveFlow: A Compact Flow-Based Model for Raw Audio](http://research.baidu.com/Blog/index-view?id=139)
- [参考资料：理解基于流的生成模型](http://nooverfit.com/wp/gan%E5%92%8Cvae%E9%83%BDout%E4%BA%86%EF%BC%9F%E7%90%86%E8%A7%A3%E5%9F%BA%E4%BA%8E%E6%B5%81%E7%9A%84%E7%94%9F%E6%88%90%E6%A8%A1%E5%9E%8B%EF%BC%88flow-based%EF%BC%89-glow%EF%BC%8Crealnvp%E5%92%8Cnice/)
- [WaveGlow: a Flow-based Generative Network for Speech Synthesis](https://github.com/NVIDIA/waveglow)

### Transformer TTS文字转语音模型
Parakeet使用PaddlePaddle动态图复现了Transformer TTS, 根据论文[Neural Speech Synthesis with Transformer Network](https://arxiv.org/abs/1809.08895)实现了基于Transformer的语音合成系统。

![file](https://pic3.zhimg.com/80/v2-63288559f14feffe67b496d8a0fd8560_720w.jpg)

在这篇论文中，作者把Transformer和Tacotron2融合，形成了TransformerTTS。
> 模型的主体还是Original Transformer，只是在输入阶段和输出阶段为了配合语音数据的特性做了改变。首先是Encoder的Input阶段，先将text逐字符转化为编号，方便Embedding，然后进入Encoder PreNet，这层网络由一个Embedding layer和三层卷积层构成，转化为512维的向量后，进入Transformer Encoder。其次是Transformer的Decoder部分，分为Input和Output。Input通过一个PreNet，将80维的梅尔声谱图转化为512维向量，这里的PreNet是一个三层的全连接网络（个人认为论文中应当解释一下为什么Encoder的PreNet是用卷积设计的，而Decoder的PreNet由全连接网络就可以解决问题）；Output部分与Tacotron2的设计完全一致。

Parakeet库中Transformer TTS支持两种vocoder，分别是Griffin-Lim algorithm和WaveFlow。WaveFlow属于基于深度神经网络的声码器，而Griffin-Lim是在仅已知幅度谱、不知道相位谱的条件下重建语音的算法，属于经典声码器，算法简单，高效。

关于模型和声码器的详细介绍请参考：
- [Transformer TTS论文地址](https://arxiv.org/abs/1809.08895)
- [基于Transformer的语音合成系统](https://zhuanlan.zhihu.com/p/66931179)
- [griffin-lim算法](https://blog.csdn.net/CSDN_71560364126/article/details/103968034)
### 数据集

本文的实验使用了LJ Speech语音数据集。这个数据集由13100句短语音组成，时长共计24小时。所有语音由同一名说话人录制，录制设备是Macbook Pro的内建麦克风。WaveFlow将所有语音信号重采样到22050kHz。

# 开发基于Parakeet模型库的脚本任务
1. Parakeet模型库文件相对较少，上传时间不会特别长
2. 运行WaveFlow的时候会遇到非常多典型的场景，如数据集解压缩、缺失依赖安装、NLTK下载网络不通等问题，本文将一一予以解决
3. WaveFlow有单GPU和多GPU的实现，目前只完成单卡训练任务，由于GPU资源紧张多卡训练任务尚未验证
4. 由于脚本任务界面较为简单，通过注释不够直观，因此将详细的“踩坑”和处理过程在本文中展开

# 第一步：下载Parakeet模型库

## Notebook环境
和PaddleDetection不同，目前Parakeet模型库目前文件还是比较少的，因此尽管在gitee上没有镜像，直接去github上拉取不需要花太长时间（一般也不会中断）

In [ ]:
# 在Notebook和本地环境，只需要一行命令就搞定了
!git clone https://github.com/PaddlePaddle/Parakeet

## 脚本任务环境
但是！在脚本任务中，需要逐个文件夹创建、逐个文件上传。并且，由于脚本任务提交时，后台实际上执行的是命令行`python -u train.py`，因此`train.py`文件是不可删除的，只能修改内容。

由于文件内容比较多，建议只上传必须的waveflow示例目录和parakeet安装必须的目录文件。

另外可以使用`print(os.getcwd())`确认一下脚本任务代码界面的路径，是`/root/paddlejob/workspace/code`

![file](https://ai-studio-static-online.cdn.bcebos.com/d231e957f77a4620b8ed8ec064da6847c54255f4fb0347019f5fac4bc36ebdc2)

![file](https://ai-studio-static-online.cdn.bcebos.com/f9546b41e1ad4aaa924b36c4d42cce16988f3fb763c047f28da26d9265b3484a)

![file](https://ai-studio-static-online.cdn.bcebos.com/3d98f5661b8a4b9a907a329fae44bacbf7221fd787e64b958b7fb76387c6f18d)


# 第二步：安装依赖库
首先，查看下官方文档里的说明
## Notebook环境

### Install libsndfile1

> Make sure the library `libsndfile1` is installed, e.g., on Ubuntu.
> ```bash
> sudo apt-get install libsndfile1
> ```

以Ubuntu操作系统为例，一定要装有`libsndfile1`这个开源声音文件格式处理库，目前AI Studio已经内置了该处理库，无需额外安装。

### Install PaddlePaddle

> Install PaddlePaddle
> See [install](https://www.paddlepaddle.org.cn/install/quick) for more details. This repo requires PaddlePaddle **1.8.2** or above.

**目前，Notebook环境支持到PaddlePaddle1.8.0，但经验证，在AI Studio的PaddlePaddle1.8.0环境WaveFlow可以正常训练。**

### Install Parakeet

> ```bash
> git clone https://github.com/PaddlePaddle/Parakeet
> cd Parakeet
> pip install -e .
> ```

### Install CMUdict for nltk

CMUdict from nltk is used to transform text into phonemes.

> ```python
> import nltk
> nltk.download("punkt")
> nltk.download("cmudict")
> ```

在Notebook环境都比较简单，照做就行。虽然`nltk.download()`会出现网络不通的报错，参考[nltk.download()出错解决](https://www.cnblogs.com/webRobot/p/6065831.html)处理即可。本文已下载并解压相关文件，可直接使用。

In [1]:
%cd Parakeet && pip install -e .

/home/aistudio/Parakeet


In [ ]:
# !unzip ../nltk_data/tokenizers/punkt.zip -d ../nltk_data/tokenizers/
# !unzip ../nltk_data/corpora/cmudict.zip -d ../nltk_data/corpora/

In [ ]:
import nltk
nltk.download("punkt")
nltk.download("cmudict")

[nltk_data] Downloading package punkt to /home/aistudio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to /home/aistudio/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True

## 脚本任务环境

在脚本任务中，Parakeet和NLTK的安装会比较麻烦

### Install libsndfile1

无需安装，下载输出的权重文件至本地预测即可。

### Install PaddlePaddle

脚本任务支持PaddlePaddle1.8.2环境，无需安装。

### Install Parakeet

此处需要将原`train.py`文件开头的`import`的最后两行注释掉，因为Parakeet未安装之前是无法`from parakeet.utils import io`的。

同时，脚本任务环境的依赖库和Notebook环境不完全一样，另外缺失了一些依赖包。在没有命令行界面的情况下，需要使用`os.system()`执行`pip`安装过程。

脚本任务环境的镜像源似乎也有点问题，因此需要额外指定国内镜像，处理代码如下：

```python
    os.system("pip install -i https://pypi.douban.com/simple/ --upgrade pip")
    os.system("pip install -i https://pypi.douban.com/simple/ unidecode==1.1.1")
    os.system("pip install -i https://pypi.douban.com/simple/ ruamel.yaml")
    os.system("pip install -i https://mirror.baidu.com/pypi/simple tensorboard")
    os.system("pip install -i https://mirror.baidu.com/pypi/simple -e .")
```

在依赖安装完成后，再`import`需要的库：

```python
    import os
    import random
    import subprocess
    import time
    from pprint import pprint
    
    import argparse
    import numpy as np
    import paddle.fluid.dygraph as dg
    from paddle import fluid
    from tensorboardX import SummaryWriter
    
    import utils
    from parakeet.utils import io
    from waveflow import WaveFlow
```

### Install CMUdict for nltk

在脚本任务中NLTK数据集也是提前上传好，只是需要移动目录到指定位置（即root目录下，`nltk.download()`在脚本任务环境的数据集默认解压目录）

![file](https://ai-studio-static-online.cdn.bcebos.com/3c3e738af72343849fc51212f4dc00e39ccfb50a0d2b4184a23410a409973301)

```python
    os.system("mv ./nltk_data /root/")
    import nltk
    nltk.download("punkt")
    nltk.download("cmudict")
```

# 第三步：准备数据集和预训练模型

参考官方文档，Parakeet项目提供了WaveFlow模型64, 96和128比特率的预训练模型

<div align="center">
<table>
    <thead>
        <tr>
            <th  style="width: 250px">
            <a href="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res64_ljspeech_ckpt_1.0.zip">WaveFlow (res. channels 64)</a>
            </th>
            <th  style="width: 250px">
            <a href="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res96_ljspeech_ckpt_1.0.zip">WaveFlow (res. channels 96)</a>
            </th>
            <th  style="width: 250px">
            <a href="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res128_ljspeech_ckpt_1.0.zip">WaveFlow (res. channels 128)</a>
            </th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>LJSpeech </th>
            <th>LJSpeech </th>
            <th>LJSpeech </th>
        </tr>
        <tr>
            <th>
            <audio src="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res64_ljspeech_samples_1.0/step_3020k_sentence_0.wav" controls="controls">
            </audio><br>
            <audio src="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res64_ljspeech_samples_1.0/step_3020k_sentence_1.wav" controls="controls">
            </audio><br>
            <audio src="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res64_ljspeech_samples_1.0/step_3020k_sentence_2.wav" controls="controls">
            </audio><br>
            <audio src="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res64_ljspeech_samples_1.0/step_3020k_sentence_3.wav" controls="controls">
            </audio><br>
            <audio src="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res64_ljspeech_samples_1.0/step_3020k_sentence_4.wav" controls="controls">
            </audio>
            </th>
            <th>
            <audio src="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res96_ljspeech_samples_1.0/step_2000k_sentence_0.wav" controls="controls">
            </audio><br>
            <audio src="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res96_ljspeech_samples_1.0/step_2000k_sentence_1.wav" controls="controls">
            </audio><br>
            <audio src="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res96_ljspeech_samples_1.0/step_2000k_sentence_2.wav" controls="controls">
            </audio><br>
            <audio src="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res96_ljspeech_samples_1.0/step_2000k_sentence_3.wav" controls="controls">
            </audio><br>
            <audio src="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res96_ljspeech_samples_1.0/step_2000k_sentence_4.wav" controls="controls">
            </audio>
            </th>
            <th>
            <audio src="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res128_ljspeech_samples_1.0/step_2000k_sentence_0.wav" controls="controls">
            </audio><br>
            <audio src="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res128_ljspeech_samples_1.0/step_2000k_sentence_1.wav" controls="controls">
            </audio><br>
            <audio src="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res128_ljspeech_samples_1.0/step_2000k_sentence_2.wav" controls="controls">
            </audio><br>
            <audio src="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res128_ljspeech_samples_1.0/step_2000k_sentence_3.wav" controls="controls">
            </audio><br>
            <audio src="https://paddlespeech.bj.bcebos.com/Parakeet/waveflow_res128_ljspeech_samples_1.0/step_2000k_sentence_4.wav" controls="controls">
            </audio>
            </th>
        </tr>
    </tbody>
</table>
</div>


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Note:** The input mel spectrogams are from validation dataset, which are not seen during training.

## Notebook环境

In [ ]:
%mkdir examples/waveflow/data

In [ ]:
!tar xjvf /home/aistudio/data/data35036/LJSpeech.tar.bz2 -C examples/waveflow/data/

In [ ]:
!unzip /home/aistudio/data/data45064/waveflow_128.zip -d  examples/waveflow/
# !unzip /home/aistudio/data/data45064/waveflow_64.zip -d  examples/waveflow/

## 脚本任务环境

在脚本任务中，默认的数据集存放目录和Notebook环境挂载的位置不同，参考示例说明
```python
# 数据集文件目录
# datasets_prefix = '/root/paddlejob/workspace/train_data/datasets/'
```

同时，由于脚本任务没有命令行界面，因此解压的时候也要用到`os.system()`，最好指定解压位置，这样后面比较好找

```python
    print('解压数据集')
    os.system("tar xjvf /root/paddlejob/workspace/train_data/datasets/data35036/LJSpeech.tar.bz2 -C ./")
    print('解压预训练模型')
    os.system("unzip /root/paddlejob/workspace/train_data/datasets/data45064/waveflow_128.zip -d /root/paddlejob/workspace/train_data/datasets/")
```

# 第四步：环境配置和YAML文件准备
## Notebook环境
1. `%set_env CUDA_VISIBLE_DEVICES=0`（Notebook环境）、`export CUDA_VISIBLE_DEVICES=0`或`os.environ['CUDA_VISIBLE_DEVICES'] = '0'`指定GPU
2. 直接修改`Parakeet/examples/waveflow/configs/waveflow_ljspeech.yaml`文件或预训练模型的`waveflow_ljspeech.yaml`文件内容，如调整learning_rate、batch_size。

In [2]:
%set_env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


## 脚本任务环境

由于脚本任务训练时执行的命令是固定的`python -u train.py`，无法输入参数，因此要提前指定` parser.add_argument()`参数解析的默认值，同时`waveflow_ljspeech.yaml`需要**离线手动调整好直接上传**。
> 注意：脚本任务不支持浏览和修改`.yml`和`.md`等格式文件。
1. `train.py`
```python
def add_options_to_parser(parser):
    parser.add_argument(
        '--model',
        type=str,
        default='waveflow',
        help="general name of the model")
    parser.add_argument(
        '--name', type=str, default='waveflow', help="specific name of the training model")
    parser.add_argument(
        '--root', type=str, default='./LJSpeech-1.1', help="root path of the LJSpeech dataset")
    # parser.add_argument(
    #     '--config', type=str, default='/root/paddlejob/workspace/train_data/datasets/waveflow_res128_ljspeech_ckpt_1.0/waveflow_ljspeech.yaml', help="config path")

    parser.add_argument(
        '--use_gpu',
        type=utils.str2bool,
        default=True,
        help="option to use gpu training")

    parser.add_argument(
        '--iteration',
        type=int,
        default=None,
        help=("which iteration of checkpoint to load, "
              "default to load the latest checkpoint"))
    parser.add_argument(
        '--checkpoint',
        type=str,
        default='/root/paddlejob/workspace/train_data/datasets/waveflow_res128_ljspeech_ckpt_1.0/step-2000000',
        help="path of the checkpoint to load")
```

2. `parakeet/utils/io.py`直接指定YAML文件的位置而不是获取`--config`参数值
```python
def add_yaml_config_to_args(config):
    """ Add args in yaml config to the args parsed by argparse. The argument in 
        yaml config will be overwritten by the same argument in argparse if they 
        are both valid.
    
    Args:
        config (args): the args returned by `argparse.ArgumentParser().parse_args()`

    Returns:
        config: the args added yaml config.
    """
    with open('./configs/waveflow_ljspeech.yaml', 'rt') as f:
        yaml_cfg = ruamel.yaml.safe_load(f)
    cfg_vars = vars(config)
    for k, v in yaml_cfg.items():
        if k in cfg_vars and cfg_vars[k] is not None:
            continue
        cfg_vars[k] = v
    return config
```

# 第五步：开始训练
## Notebook环境

In [3]:
%cd examples/waveflow

/home/aistudio/Parakeet/examples/waveflow


In [ ]:
%run train.py \
    --config=./waveflow_res128_ljspeech_ckpt_1.0/waveflow_ljspeech.yaml \
    --root=./data/LJSpeech-1.1 \
    --name=finetune \
    --checkpoint=./waveflow_res128_ljspeech_ckpt_1.0/step-2000000 \
    --batch_size=8 \
    --use_gpu=true

### 查看评估指标
1. Train-Loss

![file](https://ai-studio-static-online.cdn.bcebos.com/39503c928991410292e31be497fba4f83ed74c55ce6745e5bd35fbb0126665cd)
2. Valid-Avg-Loss 

![file](https://ai-studio-static-online.cdn.bcebos.com/f30b0cd6632748dea63aef0e1215404b37ec84e0fdbb460b835780f05ec55294)
3. Valid-1th-Flow-Log_s 

![file](https://ai-studio-static-online.cdn.bcebos.com/a3e39ffc00464848813f98066a4d41cadfe9143909b64692b856f398420a891a)



## 脚本任务环境
脚本任务中，输出目录是确定的：
```python
# 输出文件目录. 任务完成后平台会自动把该目录所有文件压缩为tar.gz包，用户可以通过「下载输出」可以将输出信息下载到本地.
# output_dir = "/root/paddlejob/workspace/output"
```

因此要注意将`train.py`中的`checkpoint directory`修改到`output_dir`中
```python
    # Make checkpoint directory.
    run_dir = os.path.join("../output", config.model, config.name)
    checkpoint_dir = os.path.join(run_dir, "checkpoint")
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
```
![file](https://ai-studio-static-online.cdn.bcebos.com/31f70a5ed71a482c92982be25a4921116004d968bbc0486aacd38b9e0170b5f4)
### 查看训练日志

脚本任务训练状态可以在【历史任务】中查看，有时可能需要排队等待计算资源
![file](https://ai-studio-static-online.cdn.bcebos.com/b0d83d5cb38e4167acf0a047371d7e84c4faca140a9747b2bc231b51fdc73541)

开始训练后可以通过日志查看训练过程

![file](https://ai-studio-static-online.cdn.bcebos.com/a6377eefa93a4209a17fef4749451bb447e74ae58d5a480aa6e008959435416e)

### 完成训练

脚本任务训练完成后，消息中心会收到一条通知

![file](https://ai-studio-static-online.cdn.bcebos.com/603891a0a9c64f66ac388573fadcd753a823eba8fd4e4752a1f8b9e65eb224a1)

如果训练输出是按照要求放在指定的 `/root/paddlejob/workspace/output`目录下，就可以直接打包下载下来。

![file](https://ai-studio-static-online.cdn.bcebos.com/4efce1ac03834e6db306f092d9eb58348521f218f0ea474e9e370794f2ef6fd7)

打开下载好的压缩包，就可以看到模型参数权重都在里面了

![file](https://ai-studio-static-online.cdn.bcebos.com/4b5e03e40ea746248084dd82bc51bbee6ea12089061a4216804a1ffb5a917a49)


# 第六步：实现文字转语音（TTS）

## 使用Transformer TTS + WaveFlow实现文字转语音

在Parakeet模型库中，Deep Voice 3、Transformer TTS或FastSpeech是TTS模型，下面演示如何使用训练好的WaveFlow模型权重作为声码器，通过Transformer TTS实现文字转语音。

Transformer TTS的训练过程与WaveFlow非常类似，本文直接使用Parakeet项目中提供的预训练模型演示效果，使用时需要同时配置Transformer TTS和WaveFlow的预训练模型权重。

在Transformer TTS的示例中，最终会将下面这段文字转为语音：

> Life was like a box of chocolates, you never know what you're gonna get.

也可以修改`Parakeet/examples/transformer_tts/synthesis.py`文件中`synthesis()`方法的输入，合成我们想要的其它英文，比如`Hello world!`

```python
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description="Synthesis model")
    add_config_options_to_parser(parser)
    args = parser.parse_args()
    # Print the whole config setting.
    pprint(vars(args))
    synthesis(
        "Hello world!",
        args)
```

In [29]:
# 切换到TransformerTTS示例
%cd /home/aistudio/Parakeet/examples/transformer_tts

/home/aistudio/Parakeet/examples/transformer_tts


In [34]:
# 下载Transformer TTS预训练模型
!wget https://paddlespeech.bj.bcebos.com/Parakeet/transformer_tts_ljspeech_ckpt_1.0.zip

In [35]:
!unzip transformer_tts_ljspeech_ckpt_1.0.zip -d transformer_tts_ljspeech_ckpt_1.0/

In [32]:
%run synthesis.py \
    --use_gpu=1 \
    --output='./synthesis' \
    --config='transformer_tts_ljspeech_ckpt_1.0/ljspeech.yaml' \
    --checkpoint_transformer='./transformer_tts_ljspeech_ckpt_1.0/step-120000' \
    --vocoder='waveflow' \
    --config_vocoder='../waveflow/waveflow_res128_ljspeech_ckpt_1.0/waveflow_ljspeech.yaml' \
    --checkpoint_vocoder='../waveflow/runs/waveflow/finetune/checkpoint/step-2010000'

{'checkpoint_transformer': './transformer_tts_ljspeech_ckpt_1.0/step-120000',
 'checkpoint_vocoder': '../waveflow/runs/waveflow/finetune/checkpoint/step-2010000',
 'config': 'transformer_tts_ljspeech_ckpt_1.0/ljspeech.yaml',
 'config_vocoder': '../waveflow/waveflow_res128_ljspeech_ckpt_1.0/waveflow_ljspeech.yaml',
 'max_len': 1000,
 'output': './synthesis',
 'stop_threshold': 0.5,
 'use_gpu': 1,
 'vocoder': 'waveflow'}
[checkpoint] Rank 0: loaded model from ./transformer_tts_ljspeech_ckpt_1.0/step-120000.pdparams
[checkpoint] Rank 0: loaded model from ../waveflow/runs/waveflow/finetune/checkpoint/step-2010000.pdparams
Synthesis completed !!!


In [33]:
import IPython
IPython.display.Audio('./synthesis/samples/waveflow.wav')

# 总结：使用脚本任务环境时，需要注意什么？

耐心！脚本任务运行的时候，输出并不像Notebook项目特别直观，因此一个脚本任务要顺利运行往往要经过不少波折，失败——提交——失败——提交，多关注日志输出是一个比较好的办法。

```python
# 日志记录. 任务会自动记录环境初始化日志、任务执行日志、错误日志、执行脚本中所有标准输出和标准出错流(例如print()),用户可以在「提交」任务后,通过「查看日志」追踪日志信息.
```
![file](https://ai-studio-static-online.cdn.bcebos.com/c633670d406e4fb0a0ce97d856bc74ff89a60637ff1d4588b10aec600c706e66)

后续更新计划：
- [Parakeet：飞桨，你是个成熟的框架了，要学会自己读论文](https://aistudio.baidu.com/aistudio/projectdetail/676162)
- 使用语音合成模型WaveFlow在AI Studio脚本任务环境多卡训练
